In [2]:
import requests
import pandas as pd

# get imdb ids

In [ ]:
# use links.csv to obtain imdb ids, then loop through some of them

In [31]:
def get_ids(n):
    dtype_dic= {'movieId': str,'imdbId' : str, "tmdbId": str}
    IDdf = pd.read_csv("data/temp_links.csv", dtype = dtype_dic)
    idlist = list(IDdf["imdbId"].head(n))
    return idlist

In [ ]:
imdbIDs = get_ids(10)

# get data from omdb

In [ ]:
#http://www.omdbapi.com/
# API-key d3de5220

In [45]:
# provide a list of datatypes that the columns shall have --> leading zeros!
'''
df_columns = ["Title", 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director','Writer','Actors','Plot','Language', \
              'Country','Awards', 'Poster', 'Ratings','Metascore','imdbRating','imdbVotes','imdbID','Type','DVD',\
              'BoxOffice','Production','Website','Response']

df_dtypes = {'Title': str,'Year' : int, "Rated": str, "Released" : str, "Runtime": int, "Genre": str, "Director": str, \
            "Writer": str, "Actors": str, "Plot": str, "Language": str, "Country": str, "Awards": str, "Poster": str, \
            "Ratings": str, "Metascore": int, "imdbRating": str, "imdbVotes": str, "imdbID": str, "Type": str, \
            "DVD": str, "BoxOffice": str, "Production": str, "Website": str, "Response": str}
'''

In [ ]:
# Receiving data from API and putting it into df
def get_data_from_omdb(imdbIDs):
    df0 = pd.DataFrame()
    for id in imdbIDs:
        url = f"http://www.omdbapi.com/?i=tt{id}&apikey=d3de5220"
        result = requests.get(url)
        j = result.json()
        df_single_movie = pd.DataFrame(j)
        df0 = pd.concat([df0, df_single_movie])
    return df0

# all required cleaning operations
def perform_cleaning(df):
    df["Released"] = pd.to_datetime(df["Released"])
    def get_mins(x):
        y = x.replace(" min", "")
        return y
    #df["Runtime"] = df["Runtime"].apply(get_mins) 
    df["Runtime"] = pd.to_numeric(df["Runtime"])
    df0 = df.drop_duplicates(keep = "first", inplace = False)
    return df0

In [ ]:
df_raw = get_data_from_omdb(imdbIDs)

In [ ]:
df_cleaned = perform_cleaning(df_raw)

In [20]:
#df0 = pd.read_csv("data/OMDB.csv")
#df0.info()

In [58]:
#df = pd.read_csv("data/OMDB.csv", dtype = df_dtypes)
#df.head(3)

In [57]:
#df.transpose()

In [54]:
#df.to_csv("data/OMDB.csv", index = False)